In [1]:
from datasets import load_dataset
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [2]:
# Загрузка токенизатора NLTK
nltk.download('punkt')

# Загрузка датасета AG News
dataset = load_dataset("ag_news")
train_data = dataset["train"]
test_data = dataset["test"]

# Токенизация текстов
X_train = [word_tokenize(text.lower()) for text in train_data["text"]]
X_test = [word_tokenize(text.lower()) for text in test_data["text"]]
y_train = train_data["label"]
y_test = test_data["label"]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\R1sed\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from gensim.models import KeyedVectors

# Загрузка GloVe (преобразуем в формат Word2Vec)
glove_path = "glove.6B.100d.txt"
glove_model = KeyedVectors.load_word2vec_format(glove_path, binary=False, no_header=True)

In [4]:
def text_to_glove_vector(tokens, model, vector_size=100):
    vector = np.zeros(vector_size)
    count = 0
    for word in tokens:
        if word in model:
            vector += model[word]
            count += 1
    if count > 0:
        vector /= count  # Усреднение
    return vector

# Преобразование текстов в векторы
X_train_glove = np.array([text_to_glove_vector(text, glove_model) for text in X_train])
X_test_glove = np.array([text_to_glove_vector(text, glove_model) for text in X_test])

In [5]:
# Обучение LogisticRegression
model = LogisticRegression(max_iter=500)
model.fit(X_train_glove, y_train)

# Предсказание и оценка
y_pred = model.predict(X_test_glove)
f1 = f1_score(y_test, y_pred, average='macro')

print(f"F1-score (GloVe + LogisticRegression): {f1:.4f}")

F1-score (GloVe + LogisticRegression): 0.8833
